<a href="https://colab.research.google.com/github/Data-to-Insight-Center/patra-toolkit/blob/main/MegaDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Patra Integration with Hugging Face

[![Run in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Data-to-Insight-Center/patra-toolkit/blob/main/examples/notebooks/HuggingFace_Example.ipynb)

---

## Table of Contents

1. [Environment Setup](#1-environment-setup)
2. [Data Loading and Preprocessing](#2-data-loading-and-preprocessing)
3. [Model Initialization and Inference](#3-model-initialization-and-inference)
4. [Model Card Generation with Patra Toolkit](#4-model-card-generation-with-patra-toolkit)
5. [Validation and Submission](#5-validation-and-submission)

---
## 1. Environment Setup

First, install and import the necessary libraries required for this project.


In [ ]:
!conda create -n pytorch_wildlife python=3.8 -y
!conda activate pytorch_wildlife
!pip install PytorchWildlife

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 820.5/820.5 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
import os
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

model = pw_detection.MegaDetectorV6(device="cpu", pretrained=True, version="MDV6-yolov10-e")
results = model.single_image_detection("/content/megadetector_test_img.JPG")

for result in results["labels"]:
  label, prob = result.split(" ")
  score = {"label": label, "probability": prob}
  print(score)

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv10x summary (fused): 503 layers, 31,589,858 parameters, 0 gradients, 169.8 GFLOPs

0: 480x640 1 animal, 2292.7ms
Speed: 16.5ms preprocess, 2292.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
{'label': 'animal', 'probability': '0.93'}


In [ ]:
import os
import torch
from huggingface_hub import hf_hub_download
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

# 1. Download the checkpoint from Hugging Face Hub
model_file = hf_hub_download(repo_id="nkarthikeyan/MegaDetectorV5", filename="md_v5a.0.0.pt")
print("Downloaded model file:", model_file)

# 2. Load the entire checkpoint from disk
checkpoint = torch.load(model_file, map_location=torch.device("cpu"))

# 3. Extract the YOLO model from the checkpoint
#    Typically, "checkpoint['model']" is a YOLO object with its own .state_dict()
if "model" in checkpoint:
    yolo_model_obj = checkpoint["model"]  # This is a YOLO object
    # Extract its state dict
    state_dict = yolo_model_obj.state_dict()
else:
    # If there's no "model" key, 'checkpoint' itself might be the YOLO model object
    yolo_model_obj = checkpoint
    state_dict = yolo_model_obj.state_dict()

# 4. Instantiate the MegaDetectorV6 architecture (with no pre-trained weights)
detection_model = pw_detection.MegaDetectorV6(device="cpu", pretrained=False, version="MDV6-yolov10-e")

# 5. Load the extracted YOLO state dictionary into the MegaDetector architecture
#    Note: If pw_detection.MegaDetectorV6 has an internal `.model` attribute that aligns with YOLO,
#    you may need to do detection_model.model.load_state_dict(state_dict, strict=False)
#    For demonstration, we assume detection_model directly accepts it:
detection_model.load_state_dict(state_dict, strict=False)
detection_model.eval()
print("Model loaded and set to evaluation mode.")

# 6. Run inference on a test image (make sure it exists at the specified path)
test_image_path = "/content/megadetector_test_img.JPG"
results = detection_model.single_image_detection(test_image_path)

# 7. Optionally, save the detection results as images
pw_utils.save_detection_images(results, "./demo_output", overwrite=True)
print("Inference completed and results saved.")


ModuleNotFoundError: No module named 'run_detector'

## 4. Model Card Generation with Patra Toolkit

Create a comprehensive Model Card to document the model's details, including metadata, bias analysis, and explainability metrics.


In [ ]:
import os
import torch
from PIL import Image
from huggingface_hub import hf_hub_download

# If your PTDetector code is in a separate file, import it:
# from your_module import PTDetector

###############################################################################
# 1. Download the MegaDetectorV5 checkpoint from Hugging Face
###############################################################################
model_file = hf_hub_download(repo_id="nkarthikeyan/MegaDetectorV5", filename="md_v5a.0.0.pt")
print("Downloaded model file:", model_file)

###############################################################################
# 2. Instantiate PTDetector with the downloaded checkpoint
#    (set force_cpu=True if you want to use CPU only)
###############################################################################
detector = PTDetector(model_path=model_file, force_cpu=True)
print("PTDetector initialized.")

###############################################################################
# 3. Run inference on a test image
###############################################################################
# Make sure you have an image at this path. Alternatively, pick any local path.
test_image_path = "/content/megadetector_test_img.JPG"

# Load the image as a PIL Image
img = Image.open(test_image_path).convert("RGB")

# Set a detection threshold (e.g., 0.2). Adjust as needed for your scenario.
detection_threshold = 0.2

# Run inference
results = detector.generate_detections_one_image(
    img_original=img,
    image_id=os.path.basename(test_image_path),
    detection_threshold=detection_threshold
)

###############################################################################
# 4. Print and/or visualize the results
###############################################################################
print("Inference results:", results)

# Optionally, if you want to save detection visualization:
# (Assuming the function single_image_detection() in PTDetector is slightly different
#  from your code, you can do it directly, or you might do your own bounding box drawing)
#
# Using your approach with pw_utils:
#
# from PytorchWildlife import utils as pw_utils
# pw_utils.save_detection_images(results, "./demo_output", overwrite=True)
# print("Inference completed and detection images saved.")


{
    "name": "Image Recognition Model using Hugging Face",
    "version": "0.1",
    "short_description": "Image recognition model for demonstration of Patra Model Cards using Hugging Face.",
    "full_description": "We have trained a deep learning model using the Hugging Face framework for image classification tasks. We use this data to run Patra model cards to capture metadata about the model.",
    "keywords": "image recognition, hugging face, patra",
    "author": "Neelesh Karthikeyan",
    "input_type": "Image",
    "category": "classification",
    "input_data": "https://huggingface.co/datasets/cifar10",
    "output_data": "https://huggingface.co/models/sachith/image_recognition_model_v01",
    "foundational_model": "None",
    "ai_model": {
        "name": "Image Recognition Hugging Face Model",
        "version": "0.1",
        "description": "Image classification model using Hugging Face Transformers and CNNs for various image recognition tasks.",
        "owner": "Neelesh Ka

## 5. Validation and Submission

After generating the Model Card, it's crucial to validate and ensure that all information is accurate and properly formatted. Submitting the Model Card allows it to be integrated into other systems or platforms as needed.

In [ ]:
# Validate the Model Card
mc.validate()

# Submit the Model Card
# mc.submit("<patra_server_url>")

# Save the Model Card to a JSON file
mc.save("huggingface_modelcard.json")
